In [1]:
import numpy as np
import datetime

Randomly generate `2^N` integer values without replacement, in order to sort on `N` CPUs.

In [2]:
np.random.seed(42)

N = 3
ARRAY_SIZE = 1 << N # 2^N
unsorted_array = np.random.choice(ARRAY_SIZE, ARRAY_SIZE, replace=False)
unsorted_array

array([1, 5, 0, 7, 2, 4, 3, 6])

1st CPU (`CPU 0`) just splits the numbers into 2 streams as they are loaded.
```
input: 1, 5, 0, 7, 2, 4, 3, 6

output:
top: 1, 0, 2, 3
bot: 5, 7, 4, 6
```

In [3]:
top_stream1 = unsorted_array[0::2]
bot_stream1 = unsorted_array[1::2]
top_stream1, bot_stream1

(array([1, 0, 2, 3]), array([5, 7, 4, 6]))

2nd CPU (`CPU 1`) reads batches from the streams of length `2^(1-1) = 1` and merge-sorts them into batches of `2^1 = 2`, which are then send in an alternating manner to the 2 output streams.
```
input:
top: 1, 0, 2, 3
bot: 5, 7, 4, 6

step 1:
1:5 -> 1
 :5 -> 5
top: 1, 5
bot:

step 2:
0:7 -> 0
 :7 -> 7
top: 1, 5
bot: 0, 7

step 3:
2:4 -> 2
 :4 -> 4
top: 1, 5, 2, 4
bot: 0, 7

step 4:
3:6 -> 3
 :6 -> 6
top: 1, 5, 2, 4
bot: 0, 7, 3, 6
```

In [4]:
top_stream2 = np.zeros_like(top_stream1)
bot_stream2 = np.zeros_like(bot_stream1)

streams2 = [top_stream2, bot_stream2]

input_batch_size = 1 << (1 - 1)
output_batch_size = 1 << (1)
input_start = 0
input_stop = input_batch_size
output_start = 0
output_stop = output_batch_size
output_stream = 0
sorted_batch = np.zeros(output_batch_size, dtype=int)

while input_start < ARRAY_SIZE >> 1:
    top = top_stream1[input_start:input_stop]
    bot = bot_stream1[input_start:input_stop]
    if top[0] < bot[0]:
        sorted_batch[0] = top[0]
        sorted_batch[1] = bot[0]
    else:
        sorted_batch[0] = bot[0]
        sorted_batch[1] = top[0]

    streams2[output_stream][output_start:output_stop] = sorted_batch
    output_stream = 1 - output_stream
    input_start += input_batch_size
    input_stop += input_batch_size
    if output_stream == 0:
        output_start += output_batch_size
        output_stop += output_batch_size

top_stream2, bot_stream2

(array([1, 5, 2, 4]), array([0, 7, 3, 6]))

3rd CPU (`CPU 2`) reads batches from the input streams of length `2^(2-1) = 2` and merge-sorts them into batches of `2^2 = 4`, which are then send in an alternating manner to the 2 output streams.
```
input:
top: 1, 5, 2, 4
bot: 0, 7, 3, 6

step 1:
0:1 -> 0
1:7 -> 1
5:7 -> 5
 :7 -> 7
top: 0, 1, 5, 7
bot: 

step 2:
2:3 -> 2
3:4 -> 3
4:6 -> 4
 :6 -> 6 
top: 0, 1, 5, 7
bot: 2, 3, 4, 6
```

In [5]:
top_stream3 = np.zeros_like(top_stream2)
bot_stream3 = np.zeros_like(bot_stream2)

streams3 = [top_stream3, bot_stream3]

input_batch_size = 1 << (2 - 1)
output_batch_size = 1 << (2)
input_start = 0
input_stop = input_batch_size
output_start = 0
output_stop = output_batch_size
output_stream = 0
sorted_batch = np.zeros(output_batch_size, dtype=int)

while input_start < ARRAY_SIZE >> 1:
    top = top_stream2[input_start:input_stop]
    bot = bot_stream2[input_start:input_stop]
    i, j, k = 0, 0, 0
    while k < output_batch_size:
        if i < input_batch_size and j < input_batch_size:
            if top[i] < bot[j]:
                sorted_batch[k] = top[i]
                i += 1
            else:
                sorted_batch[k] = bot[j]
                j += 1
        elif i < input_batch_size:
            sorted_batch[k] = top[i]
            i += 1
        else:
            sorted_batch[k] = bot[j]
            j += 1
        k += 1

    streams3[output_stream][output_start:output_stop] = sorted_batch
    output_stream = 1 - output_stream
    input_start += input_batch_size
    input_stop += input_batch_size
    if output_stream == 0:
        output_start += output_batch_size
        output_stop += output_batch_size

top_stream3, bot_stream3

(array([0, 1, 5, 7]), array([2, 3, 4, 6]))

Last CPU (`CPU 3`) reads batches from the input streams of length `2^(3-1) = 4` and merge-sorts them into a single batch of size `2^3 = 8`.
```
input:
top: 0, 1, 5, 7
bot: 2, 3, 4, 6

output:
0:2 -> 0
1:2 -> 1
5:2 -> 2
5:3 -> 3
5:4 -> 4
5:6 -> 5
7:6 -> 6
7:  -> 7
sorted_array: 0, 1, 2, 3, 4, 5, 6, 7
```

In [6]:
sorted_array = np.zeros_like(unsorted_array)

input_batch_size = 1 << (3 - 1)
output_batch_size = 1 << (3)
input_start = 0
input_stop = input_batch_size
output_start = 0
output_stop = output_batch_size
output_stream = 0
sorted_batch = np.zeros(output_batch_size, dtype=int)

while input_start < ARRAY_SIZE >> 1:
    top = top_stream3[input_start:input_stop]
    bot = bot_stream3[input_start:input_stop]
    i, j, k = 0, 0, 0
    while k < output_batch_size:
        if i < input_batch_size and j < input_batch_size:
            if top[i] < bot[j]:
                sorted_batch[k] = top[i]
                i += 1
            else:
                sorted_batch[k] = bot[j]
                j += 1
        elif i < input_batch_size:
            sorted_batch[k] = top[i]
            i += 1
        else:
            sorted_batch[k] = bot[j]
            j += 1
        k += 1

    sorted_array[output_start:output_stop] = sorted_batch
    output_stream = 1 - output_stream
    input_start += input_batch_size
    input_stop += input_batch_size
    if output_stream == 0:
        output_start += output_batch_size
        output_stop += output_batch_size

sorted_array

array([0, 1, 2, 3, 4, 5, 6, 7])

Solution for any `N`.

In [9]:
np.random.seed(int(datetime.datetime.utcnow().timestamp()))

N = 12
ARRAY_SIZE = 1 << N
unsorted_array = np.random.choice(ARRAY_SIZE, ARRAY_SIZE, replace=False)
sorted_array = None # will be generated
print(unsorted_array)

# 1st CPU
top_stream_a = unsorted_array[0::2]
bot_stream_a = unsorted_array[1::2]

top_stream_b = np.zeros_like(top_stream_a)
bot_stream_b = np.zeros_like(bot_stream_a)

streams = ((top_stream_a, bot_stream_a), (top_stream_b, bot_stream_b))

# N CPUs that can be run in a pipeline
for n in range(1, N+1):
    # ping-pong between the two streams (necessary for a single CPU implementation)
    top_stream, bot_stream = streams[~(n & 1)]
    output_streams = streams[n & 1]

    input_batch_size = 1 << (n - 1)
    output_batch_size = 1 << (n)
    input_start = 0
    input_stop = input_batch_size
    output_start = 0
    output_stop = output_batch_size
    # ping-pong between the two output streams
    output_stream = 0
    sorted_batch = np.zeros(output_batch_size, dtype=int)

    # merge sorting of the two streams using batches
    while input_start < ARRAY_SIZE >> 1:
        top = top_stream[input_start:input_stop]
        bot = bot_stream[input_start:input_stop]
        i, j, k = 0, 0, 0
        while k < output_batch_size:
            if i < input_batch_size and j < input_batch_size:
                if top[i] < bot[j]:
                    sorted_batch[k] = top[i]
                    i += 1
                else:
                    sorted_batch[k] = bot[j]
                    j += 1
            elif i < input_batch_size:
                sorted_batch[k] = top[i]
                i += 1
            else:
                sorted_batch[k] = bot[j]
                j += 1
            k += 1

        if n == N: # last CPU writes to the final sorted array
            sorted_array = sorted_batch 
        else:
            output_streams[output_stream][output_start:output_stop] = sorted_batch

        output_stream = 1 - output_stream
        input_start += input_batch_size
        input_stop += input_batch_size
        if output_stream == 0:
            output_start += output_batch_size
            output_stop += output_batch_size

print(sorted_array)
# check against numpy's sort
np.array_equal(sorted_array, np.sort(unsorted_array))

[ 194 2201 3172 ... 3412 1734 2036]
[   0    1    2 ... 4093 4094 4095]


True